In [ ]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

# Statistical Tests
from scipy.stats import f_oneway

# Preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

# Model Selection
from sklearn.cluster import KMeans
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split, GridSearchCV
from sklearn.metrics import roc_auc_score, accuracy_score, confusion_matrix, ConfusionMatrixDisplay, RocCurveDisplay, classification_report
from sklearn.feature_selection import RFECV
from sklearn.svm import SVC

# Models
import optuna
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from xgboost import XGBRegressor, XGBClassifier
from catboost import CatBoostRegressor, CatBoostClassifier
from sklearn.linear_model import Lasso, Ridge, LogisticRegression
from sklearn.ensemble import GradientBoostingRegressor, GradientBoostingClassifier
from lightgbm import LGBMClassifier, LGBMRegressor
import haversine as hs 

import warnings
warnings.filterwarnings('ignore')

import re

from sklearn.model_selection import RepeatedStratifiedKFold, StratifiedKFold, KFold, TimeSeriesSplit
from sklearn.model_selection import GroupKFold, LeaveOneGroupOut
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.ensemble import ExtraTreesRegressor, RandomForestRegressor, GradientBoostingRegressor
from sklearn.ensemble import HistGradientBoostingRegressor, VotingRegressor, StackingRegressor, AdaBoostRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import Ridge, Lasso
from sklearn.linear_model import HuberRegressor
from sklearn.cross_decomposition import PLSRegression
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error, roc_auc_score, roc_curve
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin, ClassifierMixin
from sklearn.preprocessing import FunctionTransformer, StandardScaler, MinMaxScaler, PowerTransformer
from sklearn.compose import ColumnTransformer
from sklearn.calibration import CalibratedClassifierCV
from scipy.cluster.hierarchy import dendrogram, linkage
from scipy.spatial.distance import squareform
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor

reference:

https://www.kaggle.com/code/yaaangzhou/en-playground-s3-e20-eda-modeling/notebook

In [ ]:
sample_submission = pd.read_csv('/kaggle/input/playground-series-s3e20/sample_submission.csv')
train = pd.read_csv('/kaggle/input/playground-series-s3e20/train.csv')
test = pd.read_csv('/kaggle/input/playground-series-s3e20/test.csv')

train.drop(['ID_LAT_LON_YEAR_WEEK'], axis=1, inplace=True)
test.drop(['ID_LAT_LON_YEAR_WEEK'], axis=1, inplace=True)

## 1.visualization

In [ ]:
# Latitude and longitude

fig = px.scatter_mapbox(train, lat="latitude", lon="longitude", 
                        zoom=7, height=800, width=1000,) 
fig.update_layout(mapbox_style="open-street-map", title='Map')
fig.show()

In [ ]:
train["week"] = (train["year"] - 2019) * 53 + train["week_no"]

def plot_emission(train):
    
    plt.figure(figsize=(15, 6))
    sns.lineplot(data=train, x="week", y="emission", label="Emission", alpha=0.7, color='blue')

    plt.xlabel('Week')
    plt.ylabel('Emission')
    plt.title('Emission over time')

    plt.legend()
    plt.tight_layout()
    plt.show()
    
plot_emission(train)

In [ ]:
num = train.columns.tolist()[1:-1]

corr_matrix = train[num].corr()
mask = np.triu(np.ones_like(corr_matrix, dtype=bool))

plt.figure(figsize=(15, 12))
sns.heatmap(corr_matrix, mask=mask, annot=False, cmap='Blues', fmt='.2f', linewidths=1, square=True, annot_kws={"size": 9} )
plt.title('Correlation Matrix', fontsize=15)
plt.show()

## 2.EDA

In [ ]:
# add feature
train=train[train['year']!=2020] # 删除疫情数据


train['week_no_sin'] = np.sin(train['week_no']*(2*np.pi/52))
train['week_no_cos'] = np.cos(train['week_no']*(2*np.pi/52))

test['week_no_sin'] = np.sin(test['week_no']*(2*np.pi/52))
test['week_no_cos'] = np.cos(test['week_no']*(2*np.pi/52))

In [ ]:
# 对缺失值做处理，对部分变量做scalar

def feature_preprocessing(df):

    # drop features with more than 50% nan.
    missing_ratios = df.isnull().mean()
    columns_to_drop = missing_ratios[missing_ratios > 0.5].index
    df = df.drop(columns_to_drop, axis=1)
    df = df.fillna(df.mean())

    # scalar
    sc = StandardScaler()
    for i in df.columns:
        if i not in ['week_no', 'covid_flag','latitude','longitude','emission']:
            df[i] = sc.fit_transform(df[i].values.reshape(-1,1))
    return df

train = feature_preprocessing(train)
test = feature_preprocessing(test)

In [ ]:
emission = train.pivot(index=['latitude', 'longitude'], 
                       columns=['year', 'week_no'], values='emission')
with pd.option_context("display.max_columns", 6, "display.max_rows", 6):
    display(emission)

In [ ]:
# https://www.kaggle.com/code/ambrosm/pss3e20-eda-which-makes-sense
# A PCA or an SVD of the 497 emission time series reveals that five components suffice to explain 100 % of the variance:

from sklearn.decomposition import TruncatedSVD
from matplotlib.ticker import MaxNLocator

svd = TruncatedSVD(n_components=10)
svd.fit(emission)

plt.figure(figsize=(5, 3))
plt.plot(svd.explained_variance_ratio_.cumsum(), color='g')  # explained_variance_ratio_:所选择的每个组成部分所解释的方差百分比
plt.gca().xaxis.set_major_locator(MaxNLocator(integer=True))
plt.title('Singular value decomposition of the locations')
plt.ylabel('Cumulative explained variance ratio')
plt.xlabel('Component #')
plt.show()

In [ ]:
# Clustering by emission mean
# 根据地区的排放量进行聚类
cluster_train = train.groupby(by=['latitude', 'longitude'], as_index=False)['emission'].mean()
model = KMeans(n_clusters=5)
model.fit(cluster_train)
cluster_train['kmeans_group'] = model.predict(cluster_train)

# check if emission is 0
cluster_train['is_zero'] = cluster_train['emission'].apply(lambda x: 1 if x==0 else 0)

# Distance to the highest emission location
# 找到同一地区的历史最高排放量，并计算距离它的距离 
max_lat_lon_emission = cluster_train.loc[cluster_train['emission']==cluster_train['emission'].max(), ['latitude', 'longitude']]
cluster_train['distance_to_max_emission'] = cluster_train.apply(
    lambda x: hs.haversine((x['latitude'], x['longitude']), (max_lat_lon_emission['latitude'].values[0], max_lat_lon_emission['longitude'].values[0])), axis=1)

train = train.merge(cluster_train[['latitude', 'longitude', 'kmeans_group', 'is_zero', 'distance_to_max_emission']], on=['latitude', 'longitude'])
test = test.merge(cluster_train[['latitude', 'longitude', 'kmeans_group', 'is_zero', 'distance_to_max_emission']], on=['latitude', 'longitude'])

### 3.train

In [ ]:
X = train.drop('emission',axis=1)
Y = train['emission']

# This data is for FS
# 经过预处理的数据将用在特征选择
train_fs = pd.concat([X,Y],axis = 1)

In [ ]:
# 使用同一模型XGBoost，对比不同feature下的RMSE
def calculateRMSE(x,y,features,model):
    for i,val in enumerate(x.columns):
        if val not in features:
            x = x.drop(columns=val)
    X_train, X_val, y_train, y_val = train_test_split(x,y,test_size=0.2, random_state=42)
    clf = XGBRegressor().fit(X_train,y_train)
    prediction = clf.predict(X_val)
    loss = np.sqrt(mean_squared_error(y_val,prediction))
    
    print(f"{model} has RMSE = {loss}")

In [ ]:
all_feature=train.columns.values
feature_1 = X.drop(['longitude', 'kmeans_group', 'is_zero', 'distance_to_max_emission'],axis=1).columns
feature_2 = X.drop(['longitude','distance_to_max_emission'],axis=1).columns

calculateRMSE(X,Y,all_feature,'All features')
calculateRMSE(X,Y,feature_1,'eatures 1')
calculateRMSE(X,Y,feature_2,'All features 2')

In [ ]:
# feat_cols = list(sfs_1.k_feature_idx_)
# featureSFS = X.columns[feat_cols].tolist()

featureSFS = ['latitude',
 'longitude',
 'year',
 'week_no',
 'CarbonMonoxide_sensor_azimuth_angle',
 'Cloud_solar_azimuth_angle',
 'week_no_sin',
 'kmeans_group',
 'is_zero',
 'distance_to_max_emission']

print("Selected features:", featureSFS)
calculateRMSE(X,Y,featureSFS,'Step Forward Selection')

In [ ]:
# selected_feats = ['kmeans_group', 'longitude', 'week_no', 'distance_to_max_emission', 'latitude', 'ratio_2020']
selected_feats = featureSFS
# selected_feats = featureRFE
X = train[selected_feats]
Y = Y

test = test[selected_feats]

In [ ]:
xgb_cv_scores, xgb_preds = list(), list()
lgbm_cv_scores, lgbm_preds = list(), list()
# ens_cv_scores, ens_preds = list(), list()

kf = KFold(n_splits=5, random_state=42, shuffle=True)

for i, (train_ix, test_ix) in enumerate(kf.split(X)):
    X_train, X_test = X.iloc[train_ix], X.iloc[test_ix]
    Y_train, Y_test = Y.iloc[train_ix], Y.iloc[test_ix]
    
    print('---------------------------------------------------------------')
    
    ## XGBoost
    xgb_md = XGBRegressor().fit(X_train, Y_train)
    xgb_pred = xgb_md.predict(X_test)   
    xgb_score_fold = np.sqrt(mean_squared_error(Y_test, xgb_pred))
    print('Fold', i+1, '==> XGBoost oof RMSE score is ==>', xgb_score_fold)
    xgb_cv_scores.append(xgb_score_fold)
    
    xgb_pred_test = xgb_md.predict(test)
    xgb_preds.append(xgb_pred_test)
    
    ## LGBM
    lgbm_md = LGBMRegressor(n_estimators = 1000,
                           max_depth = 15,
                           learning_rate = 0.01,
                           num_leaves = 105,
                           reg_alpha = 0.1, #0.3
                           reg_lambda = 0.1,
                           subsample = 0.7,
                           colsample_bytree = 0.8,
                           verbose=-1, # 关闭日志打印
                           ).fit(X_train, Y_train)
    lgbm_pred = lgbm_md.predict(X_test) 
    lgbm_score_fold = np.sqrt(mean_squared_error(Y_test, lgbm_pred))
    print('Fold', i+1, '==> LGBM oof RMSE score is ==>', lgbm_score_fold)
    lgbm_cv_scores.append(lgbm_score_fold)
    
    lgbm_pred_test = lgbm_md.predict(test)
    lgbm_preds.append(lgbm_pred_test) 
    
print('---------------------------------------------------------------')
print('Average RMSE of XGBoost model is:', np.mean(xgb_cv_scores))
print('Average RMSE of LGBM model is:', np.mean(lgbm_cv_scores))

In [ ]:
submission = sample_submission.copy()

submission_xgb = xgb_md.predict(test)
submission_lgb = lgbm_md.predict(test)

pre_count=(0.5*submission_xgb+0.5*submission_lgb)*1.06
submission['emission'] = pre_count

submission.to_csv('submission.csv', index=False)
